<a href="https://colab.research.google.com/github/DeependraChaddha/Heat_Equation_Solver/blob/main/Heat_Equation_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Using Gradient Descent to solve heat equation.
The difference between LHS and RHS of Heat Equation is used as Loss function.
Will experiment with optimizers to see which works best

#Make imports

In [ ]:
import torch
from torch import nn
import matplotlib

#Device-Agnostic Code

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'

Set random seeds

In [ ]:
torch.manual_seed(68)
if torch.cuda.is_available():
  torch.cuda.manual_seed(68)
  torch.cuda.manual_seed_all(68)

#Specifying equation and coordinate system

Will figure out how to solve for first example, will then functionalize and implement on example 2 and 3. Example 4 is time dependent so may have to use different approach for that

##Example 1
∂²T/∂x² + ∂²T/∂y² = 8π²sin(2πx)sin(2πy)

T(0,y) = 0
T(1,y) = 0
T(x,0) = 0
T(x,1) = 0


Take x=0,x=1, y=0,y=1 as edges because of the initial conditions

In [ ]:
#Specify number of points in grid in each direction
#todo assert both nx,ny to be strictly greater than 1 when generalizing
nx=10
ny=10

#calculate spacing between 2 points in the grid
lx=1/(nx-1)
ly=1/(ny-1)

Make grid

In [ ]:
#Make linear space of x and y and reshape to the shape(nx,1) and (ny,1) respectively (Reshaping is done so that in case of matrix multiplication or broadcasting no error is encountered)
x=torch.linspace(0,1,nx).reshape(-1,1).requires_grad_(True)
y=torch.linspace(0,1,ny).reshape(-1,1).requires_grad_(True)

#Make meshgrid
X,Y=torch.meshgrid(x.squeeze(),y.squeeze(),indexing='ij')
X.reshape(-1,1).requires_grad_(True)
Y.reshape(-1,1).requires_grad_(True)
print(X)
print(Y)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111,
         0.1111],
        [0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222,
         0.2222],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333,
         0.3333],
        [0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444,
         0.4444],
        [0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556,
         0.5556],
        [0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667,
         0.6667],
        [0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778,
         0.7778],
        [0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889,
         0.8889],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000]], grad_fn=<

In [ ]:
#To get (i,j) point
i=3
j=4
print(f'(i,j)th point is {X[i,j]},{Y[i,j]}')

(i,j)th point is 0.3333333432674408,0.4444444477558136


Make Neural Network to find solution of Heat Equation

Make Loss Function

Set Optimizer and Train


In [ ]:
#Try plotting after every few steps to visualize the training process

##Example 2
Heat Equation: ∂²T/∂x² + ∂²T/∂y² = 0

Initial Conditions:
T(0,y) = 0
T(1,y) = 0
T(x,0) = 0
T(x,1) = sin(πx)


##Example 3
Heat Equation: ∂²T/∂x² + ∂²T/∂y² = 0

Initial Conditions:
T(0,y) = 0
T(1,y) = y(1-y)
T(x,0) = 0
T(x,1) = 0


##Example 4
Heat Equation: ∂T/∂t = α(∂²T/∂x² + ∂²T/∂y²)

Initial Conditions:
T(x,y,0) = sin(πx)sin(πy)
T(0,y,t) = 0
T(1,y,t) = 0
T(x,0,t) = 0
T(x,1,t) = 0
α = 0.01
